In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
import librosa
import soundfile as sf
import torch
import pandas as pd

In [6]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.config.output_hidden_states = True

pytorch_model.bin:   1%|          | 10.5M/1.26G [00:00<?, ?B/s]

/home/mojo/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model che

In [7]:
file_path = 'path_train.txt'
with open(file_path, 'r') as file:
    audio_paths = file.readlines()

In [8]:
all_padded_sequences = [[] for _ in range(model.config.num_hidden_layers + 1)]

In [9]:
from tqdm import tqdm

# Process each audio file
for audio_path in tqdm(audio_paths):
    audio_path = audio_path.strip()
    
    # Load the audio file
    audio_input, sample_rate = sf.read(audio_path)
    
    # Process the audio input
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values
    
    # Pass through the model
    with torch.no_grad():
        outputs = model(input_values)
    
    # Get the features from all layers, including the last hidden state
    all_hidden_states = outputs.hidden_states
    
    # Store the features for each layer
    for i, layer_features in enumerate(all_hidden_states):
        all_padded_sequences[i].append(layer_features.squeeze(0))

  0%|          | 0/856 [00:00<?, ?it/s]2024-07-11 16:22:34.118780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 16:22:35.133028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
  3%|▎         | 29/856 [02:02<58:15,  4.23s/it]  


KeyboardInterrupt: 

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# Determine the maximum sequence length
max_length = max([seq.size(0) for layer in all_padded_sequences for seq in layer])

# Pad each sequence to the maximum length
padded_sequences = [pad_sequence(layer, batch_first=True, padding_value=0) for layer in all_padded_sequences]

In [ ]:
# Save each layer's padded features to a CSV file
for i, layer_features in enumerate(padded_sequences):
    # Flatten the features
    layer_flat = layer_features.view(-1, layer_features.size(-1)).numpy()
    
    # Convert to DataFrame
    df_layer = pd.DataFrame(layer_flat)
    
    # Save to CSV
    df_layer.to_csv(f"layer_{i}_features_padded.csv", index=False)


In [ ]:
layer_features.size()